In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# datasets
filepath = 'data/DisneylandReviews.csv'
disney_df = pd.read_csv(filepath, encoding='utf-8', encoding_errors='replace')

# try to read uss_df
uss_filepath = 'data/universal_studio_branches.csv'
try:
    # use the new parameter name in pandas 1.4+
    uss_df = pd.read_csv(uss_filepath,
                         encoding='utf-8',
                         encoding_errors='replace',
                         engine='python',
                         on_bad_lines='skip')  # alternative to error_bad_lines=False
except:
    try:
        # try the old parameter name
        uss_df = pd.read_csv(uss_filepath,
                             encoding='utf-8',
                             encoding_errors='replace',
                             engine='python',
                             error_bad_lines=False)
    except:
        # if all above methods fail, try to specify the separator and quote character
        uss_df = pd.read_csv(uss_filepath,
                             encoding='utf-8',
                             encoding_errors='replace',
                             engine='python',
                             sep=',',
                             quoting=3,  # QUOTE_NONE
                             escapechar='\\')  # use backslash as escape character

print(f"successfully read Universal Studios data, {len(uss_df)} rows")

# basic info
print("Disneyland Reviews:")
print(disney_df.info())
print("\nUniversal Studios Reviews:")
print(uss_df.info())

# rename columns and merge datasets
disney_renamed = disney_df.rename(columns={
    'Review_ID': 'review_id',
    'Rating': 'rating',
    'Year_Month': 'date',
    'Reviewer_Location': 'reviewer_location',
    'Review_Text': 'review_text',
    'Branch': 'branch'
})

uss_renamed = uss_df.rename(columns={
    'reviewer': 'reviewer_location',
    'written_date': 'date',
    'title': 'review_title',
    'review_text': 'review_text'
})

# tag label for disney and uss
disney_renamed['park_type'] = 'Disney'
uss_renamed['park_type'] = 'USS'

# add missing columns
if 'review_id' not in uss_renamed.columns:
    uss_renamed['review_id'] = uss_renamed.index + len(disney_renamed)
if 'review_title' not in disney_renamed.columns:
    disney_renamed['review_title'] = np.nan

# select common columns and merge datasets
common_columns = ['review_id', 'rating', 'date', 'reviewer_location', 'review_text', 'review_title', 'branch', 'park_type']
disney_common = disney_renamed[common_columns]
uss_common = uss_renamed[common_columns]

# merge datasets
combined_df = pd.concat([disney_common, uss_common], ignore_index=True)

# check merged dataset
print("\nMerged dataset info:")
print(combined_df.info())
print(combined_df.head())

successfully read Universal Studios data, 50904 rows
Disneyland Reviews:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42656 entries, 0 to 42655
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Review_ID          42656 non-null  int64 
 1   Rating             42656 non-null  int64 
 2   Year_Month         42656 non-null  object
 3   Reviewer_Location  42656 non-null  object
 4   Review_Text        42656 non-null  object
 5   Branch             42656 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.0+ MB
None

Universal Studios Reviews:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50904 entries, 0 to 50903
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   reviewer      50904 non-null  object 
 1   rating        50904 non-null  float64
 2   written_date  50904 non-null  object 
 3   title         50904 non-null  ob

In [12]:
# standardize date
def standardize_date(date_str):
    try:
        # try to parse disney format (Year_Month)
        if isinstance(date_str, str) and len(date_str.split('-')) == 2:
            year, month = date_str.split('-')
            return f"{year}-{month}-01"
        # try to parse uss format
        elif isinstance(date_str, str):
            return pd.to_datetime(date_str).strftime('%Y-%m-%d')
        else:
            return np.nan
    except:
        return np.nan

combined_df['standardized_date'] = combined_df['date'].apply(standardize_date)
combined_df['standardized_date'] = pd.to_datetime(combined_df['standardized_date'], errors='coerce')

# extract year and month
combined_df['year'] = combined_df['standardized_date'].dt.year
combined_df['month'] = combined_df['standardized_date'].dt.month

# handle missing values
print(f"Before handling missing values: \n{combined_df.isnull().sum()}")

# fill missing review titles
combined_df['review_title'] = combined_df['review_title'].fillna('No Title')

# delete rows without review text
combined_df = combined_df.dropna(subset=['review_text'])

print(f"After handling missing values: \n{combined_df.isnull().sum()}")

# convert rating to numeric
combined_df['rating'] = pd.to_numeric(combined_df['rating'], errors='coerce')

# create negative experience label (rating <= 4 is negative experience)
combined_df['bad_experience'] = (combined_df['rating'] <= 4).astype(int)

# check the distribution of negative experience
print("\nNegative experience distribution:")
print(combined_df['bad_experience'].value_counts())
print(f"Negative experience ratio: {combined_df['bad_experience'].mean():.2%}")

Before handling missing values: 
review_id                0
rating                   0
date                     0
reviewer_location        0
review_text              0
review_title         42656
branch                   0
park_type                0
standardized_date     2613
year                  2613
month                 2613
dtype: int64
After handling missing values: 
review_id               0
rating                  0
date                    0
reviewer_location       0
review_text             0
review_title            0
branch                  0
park_type               0
standardized_date    2613
year                 2613
month                2613
dtype: int64

Negative experience distribution:
bad_experience
0    51348
1    42212
Name: count, dtype: int64
Negative experience ratio: 45.12%


In [7]:
# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)

# 1. rating distribution
plt.figure(figsize=(12, 6))
sns.countplot(x='rating', hue='park_type', data=combined_df)
plt.title('Disney and USS rating distribution')
plt.xlabel('rating')
plt.ylabel('number')
plt.savefig('rating_distribution.png')
plt.close()

# 2. rating distribution of different branches
plt.figure(figsize=(14, 8))
sns.boxplot(x='branch', y='rating', hue='park_type', data=combined_df)
plt.title('rating distribution of different branches')
plt.xlabel('branch')
plt.ylabel('rating')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('branch_ratings.png')
plt.close()

# 3. rating trend over time
monthly_ratings = combined_df.groupby(['year', 'month', 'park_type'])['rating'].mean().reset_index()
monthly_ratings['date'] = pd.to_datetime(monthly_ratings[['year', 'month']].assign(day=1))

plt.figure(figsize=(16, 6))
for park in monthly_ratings['park_type'].unique():
    park_data = monthly_ratings[monthly_ratings['park_type'] == park]
    plt.plot(park_data['date'], park_data['rating'], label=park)
plt.title('rating trend over time')
plt.xlabel('date')
plt.ylabel('average rating')
plt.legend()
plt.tight_layout()
plt.savefig('rating_trend.png')
plt.close()

# 4. rating difference by different locations
top_locations = combined_df['reviewer_location'].value_counts().head(15).index
location_df = combined_df[combined_df['reviewer_location'].isin(top_locations)]

plt.figure(figsize=(14, 8))
sns.boxplot(x='reviewer_location', y='rating', data=location_df)
plt.title('rating distribution by different locations')
plt.xlabel('reviewer location')
plt.ylabel('rating')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('location_ratings.png')
plt.close()

# 5. relationship between negative experience and branches
plt.figure(figsize=(14, 6))
bad_exp_by_branch = combined_df.groupby(['branch', 'park_type'])['bad_experience'].mean().reset_index()
sns.barplot(x='branch', y='bad_experience', hue='park_type', data=bad_exp_by_branch)
plt.title('negative experience ratio by branches')
plt.xlabel('branch')
plt.ylabel('negative experience ratio')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('bad_experience_by_branch.png')
plt.close()

KeyError: 'year'

In [14]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

# check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device: {device}")

# nltk resources for sentiment analysis
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

# initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# 1. identify and use basic text features
combined_df['review_length'] = combined_df['review_text'].apply(len)
combined_df['word_count'] = combined_df['review_text'].apply(lambda x: len(str(x).split()))

# 2. sentiment analysis
combined_df['sentiment_score'] = combined_df['review_text'].apply(lambda x: sia.polarity_scores(str(x))['compound'])
combined_df['sentiment_negative'] = combined_df['review_text'].apply(lambda x: sia.polarity_scores(str(x))['neg'])
combined_df['sentiment_positive'] = combined_df['review_text'].apply(lambda x: sia.polarity_scores(str(x))['pos'])

# 3. keyword extraction - use TF-IDF to find common words in negative reviews
negative_reviews = combined_df[combined_df['bad_experience'] == 1]['review_text']
positive_reviews = combined_df[combined_df['bad_experience'] == 0]['review_text']

# TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_neg = tfidf.fit_transform(negative_reviews)
tfidf_pos = tfidf.transform(positive_reviews)

# get feature names
feature_names = tfidf.get_feature_names_out()

# calculate the average TF-IDF value of each word in negative reviews
neg_tfidf_means = tfidf_neg.mean(axis=0).A1
pos_tfidf_means = tfidf_pos.mean(axis=0).A1

# calculate the difference value (words more common in negative reviews)
diff_means = neg_tfidf_means - pos_tfidf_means

# get the top 100 words with the most distinguishing features
top_diff_indices = diff_means.argsort()[-100:]
top_neg_words = [feature_names[i] for i in top_diff_indices]

print("\nwords most distinguishing in negative reviews:")
print(top_neg_words)

# 4. create keyword features
# to improve efficiency, only use the first 20 keywords
top_20_neg_words = top_neg_words[-20:]
for word in top_20_neg_words:
    combined_df[f'contains_{word}'] = combined_df['review_text'].apply(lambda x: 1 if word.lower() in str(x).lower() else 0)

# 5. use BERT for text embedding (use all data)
def get_bert_embeddings(texts, batch_size=32):
    # ensure texts is a list of strings
    if isinstance(texts, pd.Series):
        texts = texts.astype(str).tolist()
    else:
        texts = [str(text) for text in texts]

    # load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    model.to(device)
    model.eval()

    # store the embedding vectors
    embeddings = []

    # process in batches
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]

        # tokenize and convert to tensor
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # get BERT embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # use the embedding of CLS token
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)

    return np.vstack(embeddings)

# use 10% of data for BERT embedding visualization
sample_size = max(1000, int(len(combined_df) * 0.1))
sample_df = combined_df.sample(sample_size, random_state=42)
print(f"using {sample_size} samples for BERT embedding visualization")
bert_embeddings = get_bert_embeddings(sample_df['review_text'])

# reduce the dimensionality of the BERT embeddings for visualization
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
bert_2d = pca.fit_transform(bert_embeddings)

# create visualization
plt.figure(figsize=(10, 8))
scatter = plt.scatter(bert_2d[:, 0], bert_2d[:, 1],
                     c=sample_df['bad_experience'].values,
                     cmap='coolwarm', alpha=0.6)
plt.colorbar(scatter, label='negative experience')
plt.title('visualization of BERT embeddings')
plt.xlabel('PCA dimension 1')
plt.ylabel('PCA dimension 2')
plt.savefig('bert_embeddings.png')
plt.close()

using device: cuda


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



words most distinguishing in negative reviews:
['mins', 'wouldn', 'unfortunately', 'terrible', 'waited', 'smaller', 'customer', 'fast', 'worst', 'smoking', 'having', 'expected', '90', 'need', 'entrance', 'parking', 'lot', 'know', 'limited', 'lack', 'broke', 'number', 'queuing', 'bit', 'went', 'person', 'disappointment', '45', 'maybe', 'hot', 'wasn', 'ridiculous', 'unless', 'probably', 'let', 'left', '30', 'children', 'expect', 'nice', 'open', 'bad', 'prices', 'better', 'pretty', 'service', 'pass', 'average', 'compared', 'like', 'cost', 'said', 'staff', 'paid', 'way', 'overpriced', 'just', 'did', 'attractions', 'waste', 'express', 'ok', 'wait', 'poor', 'think', 'don', 'half', 'rude', 'crowded', 'disappointing', 'didn', 'paris', 'lines', 'price', 'ticket', 'overall', 'pay', 'told', 'quite', 'ride', 'rides', 'tickets', 'queues', 'food', 'waiting', 'small', 'disappointed', 'minutes', 'expensive', 'park', 'queue', 'money', 'hours', 'hour', 'long', 'disney', 'good', 'line', 'closed', 'peopl

100%|██████████| 293/293 [04:27<00:00,  1.09it/s]


In [15]:
# 6. topic modeling - use LDA to find topics in reviews
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# create document-word matrix
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(combined_df['review_text'])
lda_feature_names = cv.get_feature_names_out()  # use a specific variable name
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)

# print the top 10 words of each topic
print("\nmain topics in reviews:")
for topic_idx, topic in enumerate(lda.components_):
    top_words_idx = topic.argsort()[:-11:-1]  # get the top 10 words

    # add safety check
    safe_indices = [i for i in top_words_idx if i < len(lda_feature_names)]

    if not safe_indices:
        print(f"    topic {topic_idx}: No valid words found")
        continue

    top_words = [lda_feature_names[i] for i in safe_indices]
    print(f"topic {topic_idx+1}: {', '.join(top_words)}")

# assign topics to each review
topic_results = lda.transform(dtm)
combined_df['dominant_topic'] = topic_results.argmax(axis=1)

# analyze the relationship between topics and negative experience
topic_bad_exp = combined_df.groupby('dominant_topic')['bad_experience'].mean().reset_index()
print("\nnegative experience ratio of each topic:")
print(topic_bad_exp)

plt.figure(figsize=(10, 6))
sns.barplot(x='dominant_topic', y='bad_experience', data=topic_bad_exp)
plt.title('negative experience ratio of each topic')
plt.xlabel('topic number')
plt.ylabel('negative experience ratio')
plt.savefig('topic_bad_experience.png')
plt.close()

# Machine Learning Models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# prepare features and target variable
# selected numeric features
numeric_features = ['review_length', 'word_count', 'sentiment_score',
                    'sentiment_negative', 'sentiment_positive']

# selected categorical features
categorical_features = ['branch', 'park_type', 'dominant_topic']

# add keyword features (use the first 20 keywords)
keyword_features = [f'contains_{word}' for word in top_20_neg_words]

# merge all features
all_features = numeric_features + categorical_features + keyword_features

# prepare data
X = combined_df[all_features].copy()
y = combined_df['bad_experience']

# handle missing values using SimpleImputer
# fill numeric features with median
num_imputer = SimpleImputer(strategy='median')
X[numeric_features] = num_imputer.fit_transform(X[numeric_features])

# fill categorical features and keyword features with mode
cat_imputer = SimpleImputer(strategy='most_frequent')
if categorical_features + keyword_features:  # ensure the list is not empty
    X[categorical_features + keyword_features] = cat_imputer.fit_transform(X[categorical_features + keyword_features])

# split training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create preprocess pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# create model pipeline using logistic regression, random forest and gradient boosting
models = {
    'Logistic Regression': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=42, max_iter=1000))
    ]),
    'Random Forest': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
    ]),
    'Gradient Boosting': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(random_state=42))
    ])
}

# train and evaluate models
results = {}
for name, model in models.items():
    print(f"\ntraining {name} model...")
    model.fit(X_train, y_train)

    # predict
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # evaluate
    print(f"{name} model evaluation results:")
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.4f}")

    # save results
    results[name] = {
        'model': model,
        'accuracy': (y_pred == y_test).mean(),
        'roc_auc': roc_auc_score(y_test, y_prob)
    }

# find the best model
best_model_name = max(results, key=lambda k: results[k]['roc_auc'])
print(f"\nbest model: {best_model_name}")
print(f"ROC AUC: {results[best_model_name]['roc_auc']:.4f}")


main topics in reviews:
topic 1: park, pass, day, ride, rides, time, wait, line, express, fast
topic 2: ride, universal, rides, park, potter, harry, studios, great, fun, day
topic 3: rides, day, park, great, time, kids, food, place, good, fun
topic 4: disney, park, rides, disneyland, parks, paris, time, just, ride, food
topic 5: disneyland, disney, park, time, like, place, just, visit, day, world

negative experience ratio of each topic:
   dominant_topic  bad_experience
0               0        0.623487
1               1        0.383549
2               2        0.340998
3               3        0.675401
4               4        0.316063

training Logistic Regression model...
Logistic Regression model evaluation results:
              precision    recall  f1-score   support

           0       0.69      0.77      0.73     10232
           1       0.68      0.59      0.63      8480

    accuracy                           0.69     18712
   macro avg       0.69      0.68      0.68     18

In [ ]:
# BERT model training
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# prepare data
texts = combined_df['review_text'].tolist()
labels = combined_df['bad_experience'].tolist()

# split training and test set
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

print(f"training set size: {len(train_texts)}, test set size: {len(test_texts)}")

# load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenize texts - use batch processing to save memory
def tokenize_in_batches(texts, batch_size=1000):
    # ensure texts is a list of strings
    if isinstance(texts, pd.Series):
        texts = texts.astype(str).tolist()
    else:
        texts = [str(text) for text in texts]

    all_input_ids = []
    all_attention_masks = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        encodings = tokenizer(batch_texts, truncation=True, padding=True, max_length=128)
        all_input_ids.extend(encodings['input_ids'])
        all_attention_masks.extend(encodings['attention_mask'])

    return {
        'input_ids': all_input_ids,
        'attention_mask': all_attention_masks
    }

print("tokenizing training set...")
train_encodings = tokenize_in_batches(train_texts)
print("tokenizing test set...")
test_encodings = tokenize_in_batches(test_texts)

# convert to PyTorch tensors
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels)
)

# create data loader - use larger batch size to accelerate training
batch_size = 32  # adjust based on GPU memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

# set optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# train model
model.train()
num_epochs = 3  # adjust based on dataset size and time limit

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    total_loss = 0
    for batch in tqdm(train_loader):
        # move data to device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # forward propagation
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"average loss: {total_loss / len(train_loader):.4f}")

# evaluate model
model.eval()
predictions = []
true_labels = []
all_probs = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        probs = F.softmax(logits, dim=1)
        all_probs.extend(probs[:, 1].cpu().numpy())

        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

# calculate evaluation metrics
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

print("\nBERT model evaluation results:")
print(f"accuracy: {accuracy_score(true_labels, predictions):.4f}")
print(f"ROC AUC: {roc_auc_score(true_labels, all_probs):.4f}")
print(classification_report(true_labels, predictions))

# Complaint Severity Analysis
def classify_complaint_severity(text, rating, sentiment_neg, sentiment_score):
    """
    classify complaint severity based on review text, rating and sentiment analysis

    returns:
    0 - no complaint
    1 - mild complaint
    2 - moderate complaint
    3 - severe complaint
    """
    if rating > 4:  # high rating is not usually a complaint
        return 0

    # use sentiment analysis
    neg_score = sentiment_neg
    compound_score = sentiment_score

    # determine severity based on rating, negative sentiment and compound sentiment
    if rating <= 2 and (neg_score > 0.5 or compound_score < -0.5):
        return 3  # severe complaint
    elif rating <= 3 and (neg_score > 0.3 or compound_score < -0.3):
        return 2  # moderate complaint
    elif rating <= 4:
        return 1  # mild complaint
    else:
        return 0  # no complaint

# apply complaint severity classification
combined_df['complaint_severity'] = combined_df.apply(
    lambda row: classify_complaint_severity(
        row['review_text'],
        row['rating'],
        row['sentiment_negative'],
        row['sentiment_score']
    ), axis=1
)

# analyze complaint severity distribution
severity_counts = combined_df['complaint_severity'].value_counts().sort_index()
print("\ncomplaint severity distribution:")
print(severity_counts)
print(f"complaint ratio: {(combined_df['complaint_severity'] > 0).mean():.2%}")

# visualize complaint severity distribution
plt.figure(figsize=(10, 6))
sns.countplot(x='complaint_severity', hue='park_type', data=combined_df)
plt.title('complaint severity distribution')
plt.xlabel('severity level (0=no complaint, 3=severe complaint)')
plt.ylabel('number')
plt.savefig('complaint_severity_distribution.png')
plt.close()

# Recovery Strategy
def recommend_recovery_action(severity, topic, branch):
    """
    recommend recovery strategy based on complaint severity, topic and branch
    """
    recovery_actions = {
        0: "no action needed",
        1: "send apology email and provide small discount coupon",
        2: "customer service team to contact and provide partial refund or free upgrade",
        3: "management intervention, full refund or free re-visit, and root cause analysis"
    }

    # customize recovery strategy based on topic (assume topic meaning as follows)
    topic_specific = {
        0: "facility maintenance and cleaning",
        1: "employee training and customer service",
        2: "queue and capacity management",
        3: "food quality and variety",
        4: "information provision and expectation management"
    }

    base_action = recovery_actions[severity]
    if severity > 0 and topic in topic_specific:
        return f"{base_action}, focus on {topic_specific[topic]}"
    return base_action

combined_df['recommended_action'] = combined_df.apply(
    lambda row: recommend_recovery_action(
        row['complaint_severity'],
        row['dominant_topic'],
        row['branch']
    ), axis=1
)

# analyze recommended recovery strategy distribution
action_counts = combined_df['recommended_action'].value_counts()
print("\nrecommended recovery strategy distribution:")
print(action_counts)

# summarize results
print("\ncomplaint prediction and recovery analysis summary:")
print(f"1. total comments: {len(combined_df)}")
print(f"2. negative experience ratio: {combined_df['bad_experience'].mean():.2%}")
print(f"3. complaint severity distribution: {severity_counts.to_dict()}")
print(f"4. best prediction model: {best_model_name}, ROC AUC: {results[best_model_name]['roc_auc']:.4f}")

training set size: 74848, test set size: 18712
tokenizing training set...


100%|██████████| 75/75 [02:36<00:00,  2.08s/it]


tokenizing test set...


100%|██████████| 19/19 [00:37<00:00,  1.95s/it]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


100%|██████████| 2339/2339 [24:18<00:00,  1.60it/s]


average loss: 0.4701
Epoch 2/3


100%|██████████| 2339/2339 [24:27<00:00,  1.59it/s]


average loss: 0.3985
Epoch 3/3


 62%|██████▏   | 1443/2339 [15:05<09:20,  1.60it/s]